# Task 5D – Teil 2: Datenvorbereitung & Äußerungsextraktion

**Umfang dieses Notebooks (Teil 2):**

1. Laden und Parsen der `spk_*.vtt`-Transkriptdateien
2. Extraktion von Sprecher-IDs, Zeitstempeln und Äußerungstexten pro Session
3. Laden der jeweiligen `speaker_to_cluster.json`-Dateien
4. Verknüpfung der Äußerungen mit den zugehörigen Sprecher-Cluster-Zuordnungen
5. Persistieren der bereinigten Daten als `.parquet`-Dateien für schnelle Wiederverwendung

> **Hinweis:** Dieses Notebook setzt die abgeschlossenen Schritte aus Teil 1 voraus (Daten-Download & -Entpackung). Es bereitet die Datengrundlage für semantisches Feature Engineering in Teil 3.


## 1) Bibliotheken laden & Konfiguration vorbereiten

In diesem Schritt werden die benötigten Bibliotheken geladen und zentrale Parameter aus der Datei `run_config.json` eingelesen.

Dazu gehören insbesondere:

* das Projektverzeichnis (`PROJECT_ROOT`)
* die Pfade zu den entpackten Rohdaten (`raw/train`, `raw/dev`)
* vorbereitende Strukturen zur weiteren Verarbeitung

> Ziel ist es, die im Setup definierte Umgebung reproduzierbar weiterzuverwenden und eine einheitliche Grundlage für die Verarbeitung der VTT- und JSON-Dateien zu schaffen.


In [1]:
#!pip install -q --user webvtt-py


In [2]:
# -----------------------------------------------
# 1) Imports und Laden der Konfiguration
# -----------------------------------------------

# Standardbibliotheken
import json                     # Zum Laden der gespeicherten Konfiguration (run_config.json)
from pathlib import Path        # Plattformunabhängige Arbeit mit Dateipfaden

# Datenverarbeitung
import pandas as pd             # Für spätere Verarbeitung und Speicherung als DataFrame/Parquet

from tqdm import tqdm  # Fortschrittsbalken für Schleifen

# -----------------------------------------------------
# Projektverzeichnis definieren (wie in Teil 1 bekannt)
# -----------------------------------------------------
# Hinweis: Bei Bedarf anpassen, falls du in einem anderen Verzeichnis arbeitest.
PROJECT_ROOT = Path.home() / "AUVIS/task5D_ml_prototype"

# Pfad zur Konfigurationsdatei (wurde in Teil 1 erzeugt)
cfg_path = PROJECT_ROOT / "run_config.json"

# -----------------------------------------------------
# Konfigurationsdatei einlesen und als Dictionary laden
# -----------------------------------------------------
with open(cfg_path, "r", encoding="utf-8") as f:
    cfg = json.load(f)

# -----------------------------------------------------
# Pfade zu den entpackten Trainings- und Entwicklungsdaten
# -----------------------------------------------------
RAW_TRAIN = Path(cfg["data"]["raw_train"])
RAW_DEV   = Path(cfg["data"]["raw_dev"])

# Ausgabe zur Kontrolle
print("Train-Verzeichnis:", RAW_TRAIN)
print("Dev-Verzeichnis:  ", RAW_DEV)


Train-Verzeichnis: /home/ercel001/AUVIS/task5D_ml_prototype/data/raw/train
Dev-Verzeichnis:   /home/ercel001/AUVIS/task5D_ml_prototype/data/raw/dev


## 2) VTT-Dateien parsen

In diesem Schritt werden alle `spk_*.vtt`-Dateien pro Session eingelesen und geparst.
Diese enthalten Zeitstempel sowie die zugehörigen Transkripte einzelner Sprecheräußerungen.

Ziel ist es, pro Datei eine strukturierte Tabelle mit folgenden Spalten zu erzeugen:

* `session_id`: Sitzungskennung (abgeleitet aus dem Dateipfad)
* `speaker_id`: Kennung des Sprechers (abgeleitet aus dem Dateinamen)
* `start_s`: Startzeitpunkt der Äußerung (in Sekunden)
* `end_s`: Endzeitpunkt der Äußerung (in Sekunden)
* `text`: Transkribierter Text der Äußerung

Diese Daten bilden die Grundlage für die spätere Verknüpfung mit den Cluster-Zuordnungen.


In [3]:
# ---------------------------------------------
# 2) VTT-Dateien parsen
# ---------------------------------------------
# Ziel dieses Blocks:
# - Einzelne WebVTT-Dateien (spk_*.vtt) einlesen
# - Zeitstempel, Sprecher-ID und Text extrahieren
# - Ergebnis als strukturierter DataFrame zurückgeben

import re
import webvtt  # Bibliothek zum Einlesen von .vtt-Dateien (Web Video Text Tracks)

def infer_session_id(path: Path) -> str:
    """
    Ermittelt die Session-ID robust aus dem Dateipfad.
    Typische Struktur: .../session_XX/labels/spk_003.vtt
    - Bevorzugt den nächstgelegenen Ordnernamen, der mit 'session' beginnt
      und Ziffern enthält (z. B. 'session_00', 'session_132').
    - Fallback: eine Ebene über 'labels' bzw. parent-of-parent.
    """
    # Kandidaten: vom Dateistandort nach oben laufen
    for p in [path.parent, *path.parents]:
        name = p.name.lower()
        if name.startswith("session") and any(ch.isdigit() for ch in name):
            return p.name  # originaler Name, nicht lowercased
    # Falls kein 'session_*' gefunden wurde: eine Ebene höher nehmen, wenn vorhanden
    return path.parents[1].name if len(path.parents) > 1 else path.parent.name

def infer_speaker_id(path: Path) -> str:
    """
    Extrahiert die Sprecher-ID robust aus dem Dateinamen.
    Beispiele:
      - 'spk_003.vtt' -> '003'
      - 'spk_5.vtt'   -> '5'
      - '003.vtt'     -> '003'
    Fallback: kompletter Stem, falls keine Ziffern gefunden werden.
    """
    stem = path.stem  # Dateiname ohne Suffix
    m = re.search(r'(\d+)', stem)
    if m:
        return m.group(1)
    # falls das Schema 'spk_xxx' ohne Ziffern wäre (unwahrscheinlich), Prefix entfernen
    return stem.replace("spk_", "").strip()

def parse_vtt_file(path: Path) -> pd.DataFrame:
    """
    Liest eine VTT-Datei ein und gibt ein DataFrame mit standardisierten Spalten zurück:
      - session_id: Sitzungskennung (aus Pfad ermittelt, z. B. 'session_00')
      - speaker_id: Sprecherkennung (aus Dateiname ermittelt)
      - utt_id:     eindeutige Äußerungs-ID (kombiniert aus session, speaker, index)
      - start_s / end_s: Start- und Endzeit (Sekunden)
      - text:       Transkribierter Text der Äußerung
    """
    utts = []

    session_id = infer_session_id(path)
    speaker_id = infer_speaker_id(path)

    # Jede Caption im VTT durchlaufen (eine Caption = eine Äußerung)
    for i, caption in enumerate(webvtt.read(path)):
        # caption.text kann Zeilenumbrüche enthalten → trimmen
        txt = caption.text.strip() if hasattr(caption, "text") else ""
        utts.append({
            "session_id": session_id,
            "speaker_id": speaker_id,
            "utt_id": f"{session_id}_{speaker_id}_{i:04d}",
            "start_s": caption.start_in_seconds,
            "end_s": caption.end_in_seconds,
            "text": txt,
        })

    return pd.DataFrame(utts)


## 3) Laden und Zusammenführen der VTT-Dateien je Split

Alle WebVTT-Dateien (`spk_*.vtt`) werden pro Split (Train / Dev) rekursiv eingelesen, geparst und in einem einheitlichen DataFrame zusammengeführt.
Dies ermöglicht eine konsistente Weiterverarbeitung der Sprecheräußerungen über alle Sitzungen hinweg.



In [4]:
# ---------------------------------------------
# 3) VTTs je Split laden und kombinieren
# ---------------------------------------------
# Ziel dieses Blocks:
# - Alle VTT-Dateien für einen Split (z. B. train oder dev) rekursiv finden
# - Jede Datei parsen (parse_vtt_file)
# - Konsolidierten DataFrame für den gesamten Split erzeugen

from tqdm import tqdm  # Fortschrittsbalken für Schleifen

def load_vtts(split_dir: Path) -> pd.DataFrame:
    """
    Lädt alle VTT-Dateien im angegebenen Verzeichnis (z. B. raw/train oder raw/dev),
    parst sie einzeln mit der Funktion `parse_vtt_file`, und gibt einen kombinierten DataFrame zurück.

    Parameter:
    ----------
    split_dir : Path
        Wurzelverzeichnis des jeweiligen Splits, z. B. ".../data/raw/train"

    Rückgabe:
    ---------
    pd.DataFrame
        Konsolidierter DataFrame mit allen Sprecheräußerungen dieses Splits.
    """
    all_utts = []

    # Alle passenden VTT-Dateien rekursiv sammeln (häufig liegen sie unter .../session_XX/labels/spk_*.vtt)
    vtt_files = list(split_dir.rglob("spk_*.vtt"))
    if not vtt_files:
        print(f"⚠️ Keine VTT-Dateien unter {split_dir} gefunden (Pattern 'spk_*.vtt').")
        return pd.DataFrame(columns=["session_id","speaker_id","utt_id","start_s","end_s","text"])

    for vtt_path in tqdm(vtt_files, desc=f"Parsing {split_dir.name}"):
        df = parse_vtt_file(vtt_path)
        if not df.empty:
            all_utts.append(df)

    if not all_utts:
        print(f"⚠️ Keine Utterances aus {split_dir} extrahiert.")
        return pd.DataFrame(columns=["session_id","speaker_id","utt_id","start_s","end_s","text"])

    return pd.concat(all_utts, ignore_index=True)

# ---------------------------
# Anwendung auf beide Splits
# ---------------------------
df_train_utts = load_vtts(RAW_TRAIN)
df_dev_utts   = load_vtts(RAW_DEV)

# Ergebnisübersicht
print("Train:", df_train_utts.shape)
print("Dev:  ", df_dev_utts.shape)

# Schneller Plausibilitätscheck: Haben wir jetzt mehrere Sessions?
print("\nTrain – Anzahl Sessions:", df_train_utts["session_id"].nunique())
print(df_train_utts["session_id"].value_counts().head(10))

print("\nDev – Anzahl Sessions:", df_dev_utts["session_id"].nunique())
print(df_dev_utts["session_id"].value_counts().head(10))


Parsing dev: 100%|██████████| 139/139 [00:00<00:00, 255.93it/s]

Train: (19076, 6)
Dev:   (8561, 6)

Train – Anzahl Sessions: 56
session_id
session_84    539
session_58    458
session_62    453
session_86    450
session_24    445
session_25    439
session_85    439
session_22    433
session_83    430
session_59    428
Name: count, dtype: int64

Dev – Anzahl Sessions: 25
session_id
session_132    407
session_43     405
session_40     401
session_44     397
session_42     389
session_136    387
session_133    381
session_134    362
session_137    356
session_55     354
Name: count, dtype: int64


In [5]:
print("Train – Anzahl Sessions:", df_train_utts["session_id"].nunique())
print(df_train_utts["session_id"].value_counts().head(10))

print("\nDev – Anzahl Sessions:", df_dev_utts["session_id"].nunique())
print(df_dev_utts["session_id"].value_counts().head(10))


Train – Anzahl Sessions: 56
session_id
session_84    539
session_58    458
session_62    453
session_86    450
session_24    445
session_25    439
session_85    439
session_22    433
session_83    430
session_59    428
Name: count, dtype: int64

Dev – Anzahl Sessions: 25
session_id
session_132    407
session_43     405
session_40     401
session_44     397
session_42     389
session_136    387
session_133    381
session_134    362
session_137    356
session_55     354
Name: count, dtype: int64


## 4) Laden der Cluster-Zuordnung und Verknüpfung mit den Äußerungen

Für jede Sitzung wird die Datei `speaker_to_cluster.json` geladen, welche die Zuordnung einzelner Sprecher-IDs zu Konversations-Clustern enthält.
Die Zuordnungen werden mit den zuvor extrahierten Äußerungen (`spk_*.vtt`) pro Session zusammengeführt, sodass jede Äußerung einem spezifischen Gesprächscluster zugewiesen werden kann.



In [6]:
# ---------------------------------------------
# 4) Laden der Cluster-Zuordnung und Verknüpfung mit den Äußerungen
# ---------------------------------------------
# Ziel dieses Blocks:
# - Für jede Session die Datei speaker_to_cluster.json laden
# - Diese enthält ein Dictionary: {speaker_id: cluster_id}
# - Alle Zuordnungen in ein globales Mapping überführen: (session_id, speaker_id) → cluster_id
# - Dieses Mapping auf alle Äußerungen anwenden, um jeder Zeile im DataFrame einen Cluster zuzuweisen

def load_clusters(split_dir: Path):
    """
    Lädt alle speaker_to_cluster.json-Dateien im angegebenen Split-Verzeichnis
    und baut ein Mapping (session_id, speaker_id) → cluster_id auf.
    """
    mapping = {}

    for json_path in split_dir.rglob("speaker_to_cluster.json"):
        with open(json_path, "r", encoding="utf-8") as f:
            session_mapping = json.load(f)

            # Session-ID korrekt aus Ordnerstruktur ziehen
            # typischer Pfad: .../session_132/labels/speaker_to_cluster.json
            if json_path.parent.name == "labels":
                session_id = json_path.parent.parent.name
            else:
                session_id = json_path.parent.name

            for spk_id, cluster in session_mapping.items():
                mapping[(session_id, spk_id)] = cluster

    return mapping

# Cluster-Mappings separat für Trainings- und Dev-Split laden
cluster_train = load_clusters(RAW_TRAIN)
cluster_dev   = load_clusters(RAW_DEV)

# ---------------------------------------------
# Cluster-Zuordnung auf die Äußerungs-DataFrames anwenden
# ---------------------------------------------

def assign_cluster(df, cluster_map):
    """
    Ergänzt das DataFrame um eine neue Spalte 'cluster',
    basierend auf dem Mapping (session_id, speaker_id) → cluster_id.
    Wenn keine Zuordnung existiert, wird -1 eingetragen.
    """
    return df.assign(
        cluster=df.apply(
            lambda row: cluster_map.get((row["session_id"], row["speaker_id"]), -1),
            axis=1
        )
    )

# Anwendung der Funktion auf Trainings- und Dev-Daten
df_train_utts = assign_cluster(df_train_utts, cluster_train)
df_dev_utts   = assign_cluster(df_dev_utts, cluster_dev)


## 5) Persistenz der Äußerungsdaten als Parquet-Dateien

Die verarbeiteten und mit Clustern angereicherten DataFrames werden im **Apache Parquet-Format** gespeichert.
Dieses Format ermöglicht eine **kompakte Speicherung**, **schnelles Laden** sowie **spaltenbasierten Zugriff** und ist daher ideal für die weitere Verarbeitung in Notebook 3 und darüber hinaus.

Die Speicherung erfolgt jeweils getrennt für Trainings- und Entwicklungsdaten (`train.parquet` / `dev.parquet`).

In [7]:
# ---------------------------------------------
# 5) Persistenz der Äußerungsdaten als Parquet
# ---------------------------------------------
# Ziel dieses Blocks:
# - Speichern der train/dev-DataFrames mit Sprecheräußerungen + Cluster-Zuordnung
# - Format: Apache Parquet (kompakt, performant, spaltenbasiert)
# - Speicherort: ~/AUVIS/task5D_ml_prototype/data/prepared/

# Zielverzeichnis vorbereiten
out_dir = PROJECT_ROOT / "data" / "prepared"
out_dir.mkdir(parents=True, exist_ok=True)  # idempotent: erstellt auch Zwischenordner

# Neue, saubere Multi-Session-Dateien speichern
out_train = out_dir / "train_utterances_multisession.parquet"
out_dev   = out_dir / "dev_utterances_multisession.parquet"

df_train_utts.to_parquet(out_train, index=False)
df_dev_utts.to_parquet(out_dev, index=False)

# Übersicht der gespeicherten Dateien ausgeben
print("Gespeichert:")
for f in [out_train, out_dev]:
    print(" -", f.name, f"({f.stat().st_size/1e6:.2f} MB)")


Gespeichert:
 - train_utterances_multisession.parquet (0.58 MB)
 - dev_utterances_multisession.parquet (0.30 MB)


### ✅ Nächste Schritte (für Teil 3)

Fahre mit dem nächsten Notebook **Teil 3: Feature Engineering & Embedding-Generierung** fort, sobald die Parquet-Dateien erfolgreich gespeichert wurden:

* Extrahieren semantischer Merkmale aus dem Äußerungstext mittels vortrainierter Sentence-Embedding-Modelle
* Persistieren der Embeddings (pro Äußerung) zur späteren Nutzung im Clustering
* Vorbereitung weiterführender Merkmale (optional): prosodisch, visuell, kontextuell
* Speicherung als strukturierte Feature-Dateien für Teil 4 (Clustering & Analyse)
